In [1]:
import pandas as pd
from sklearn import tree
import pydotplus
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
import numpy as np

In [2]:
# Import the datasets 
senseData = pd.read_csv("Lerror1.csv", header = None)
senseData.columns = ["Case_ID", "Timestamp", "Type", "Sensor"]
senseData

,Case_ID,Timestamp,Type,Sensor
0,2022-09-02,2022-09-02T08:10:53.907781562,PresenceSensor,door_C
1,2022-09-02,2022-09-02T08:10:57.529101905,FloorSensor,sens4_C
2,2022-09-02,2022-09-02T08:13:30.564635810,FloorSensor,sens4_C
3,2022-09-02,2022-09-02T08:16:01.771742591,FloorSensor,sens4_C
4,2022-09-02,2022-09-02T08:16:04.600169715,FloorSensor,sens2_C
...,...,...,...,...
3617,2022-12-10,2022-12-10T08:13:12.042216496,FloorSensor,sens3_C
3618,2022-12-10,2022-12-10T08:13:42.042216496,FloorSensor,sens3_C
3619,2022-12-10,2022-12-10T08:14:12.042216496,FloorSensor,sens3_C
3620,2022-12-10,2022-12-10T08:14:42.042216496,FloorSensor,sens3_C


In [3]:
newdataset = pd.DataFrame(columns =["Case_ID", "Timestamp", "Type", "Sensor", "Label", "Activity_ID"])
temp = 0
i = 0

for index, row in senseData.iterrows():
    activity = row["Sensor"][-1]
    new_row = {}
    new_row["Case_ID"] = row["Case_ID"]
    new_row["Timestamp"] = row["Timestamp"]
    new_row["Type"] = row["Type"]
    new_row["Sensor"] = row["Sensor"]
    # First row of the dataset
    if(temp == 0 and i == 0):
        new_row["Label"] = activity
        new_row["Activity_ID"] = i
        temp = activity
    # if the actual row has the same label of the stored, then is the same group
    elif(activity == temp):
        new_row["Activity_ID"] = i
        new_row["Label"] = activity
    # if the label mismatch, start a new group
    elif(activity != temp):
        i = i+1
        temp = activity
        new_row["Activity_ID"] = i
        new_row["Label"] = activity
    newdataset = newdataset.append(new_row, ignore_index=True)
newdataset

,Case_ID,Timestamp,Type,Sensor,Label,Activity_ID
0,2022-09-02,2022-09-02T08:10:53.907781562,PresenceSensor,door_C,C,0
1,2022-09-02,2022-09-02T08:10:57.529101905,FloorSensor,sens4_C,C,0
2,2022-09-02,2022-09-02T08:13:30.564635810,FloorSensor,sens4_C,C,0
3,2022-09-02,2022-09-02T08:16:01.771742591,FloorSensor,sens4_C,C,0
4,2022-09-02,2022-09-02T08:16:04.600169715,FloorSensor,sens2_C,C,0
...,...,...,...,...,...,...
3617,2022-12-10,2022-12-10T08:13:12.042216496,FloorSensor,sens3_C,C,220
3618,2022-12-10,2022-12-10T08:13:42.042216496,FloorSensor,sens3_C,C,220
3619,2022-12-10,2022-12-10T08:14:12.042216496,FloorSensor,sens3_C,C,220
3620,2022-12-10,2022-12-10T08:14:42.042216496,FloorSensor,sens3_C,C,220


In [4]:
# Convert the date columns in datetime type
newdataset['Timestamp'] = pd.to_datetime(newdataset['Timestamp'], format="%Y-%m-%dT%H:%M:%S.%f")
newdataset

,Case_ID,Timestamp,Type,Sensor,Label,Activity_ID
0,2022-09-02,2022-09-02 08:10:53.907781562,PresenceSensor,door_C,C,0
1,2022-09-02,2022-09-02 08:10:57.529101905,FloorSensor,sens4_C,C,0
2,2022-09-02,2022-09-02 08:13:30.564635810,FloorSensor,sens4_C,C,0
3,2022-09-02,2022-09-02 08:16:01.771742591,FloorSensor,sens4_C,C,0
4,2022-09-02,2022-09-02 08:16:04.600169715,FloorSensor,sens2_C,C,0
...,...,...,...,...,...,...
3617,2022-12-10,2022-12-10 08:13:12.042216496,FloorSensor,sens3_C,C,220
3618,2022-12-10,2022-12-10 08:13:42.042216496,FloorSensor,sens3_C,C,220
3619,2022-12-10,2022-12-10 08:14:12.042216496,FloorSensor,sens3_C,C,220
3620,2022-12-10,2022-12-10 08:14:42.042216496,FloorSensor,sens3_C,C,220


In [5]:
out = pd.DataFrame(columns = ["Case_ID", "Timestamp", "Type", "Sensor", "Label", "Activity_ID"])
i = 0
for idc, case in newdataset.groupby("Case_ID"):
    for id, window in case.groupby(pd.Grouper(freq="30s", key = "Timestamp")):
        if(window.empty):
            pass
        else:
            most_freq_label = window['Label'].value_counts().idxmax()
            dict = {"Case_ID":[],"Timestamp":[],"Type":[],"Sensor":[],"Label":[],"Activity_ID":[]}
            for a, el in window.iterrows():
                dict.get("Case_ID").append(el.get("Case_ID"))
                dict.get("Timestamp").append(el.get("Timestamp"))
                dict.get("Type").append(el.get("Type"))
                dict.get("Sensor").append(el.get("Sensor"))
                dict.get("Label").append(most_freq_label)
                dict.get("Activity_ID").append(i)
            i = i+1
            df2 = pd.DataFrame(dict)
            out = pd.concat([out, df2], ignore_index = True)
            # if(i==20):
            #     break

In [6]:
out

,Case_ID,Timestamp,Type,Sensor,Label,Activity_ID
0,2022-09-02,2022-09-02 08:10:53.907781562,PresenceSensor,door_C,C,0
1,2022-09-02,2022-09-02 08:10:57.529101905,FloorSensor,sens4_C,C,0
2,2022-09-02,2022-09-02 08:13:30.564635810,FloorSensor,sens4_C,C,1
3,2022-09-02,2022-09-02 08:16:01.771742591,FloorSensor,sens4_C,C,2
4,2022-09-02,2022-09-02 08:16:04.600169715,FloorSensor,sens2_C,C,2
...,...,...,...,...,...,...
3617,2022-12-10,2022-12-10 08:13:12.042216496,FloorSensor,sens3_C,C,2992
3618,2022-12-10,2022-12-10 08:13:42.042216496,FloorSensor,sens3_C,C,2993
3619,2022-12-10,2022-12-10 08:14:12.042216496,FloorSensor,sens3_C,C,2994
3620,2022-12-10,2022-12-10 08:14:42.042216496,FloorSensor,sens3_C,C,2995


In [7]:
# Save the file
out.to_csv("Lerror1_labeled.csv", index=False)